# 필요한 라이브러리 임포트

In [88]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd

# 전처리 함수 정의

In [2]:
# 문자 종류 확인
def build_vocab_from_df(df: pd.DataFrame):
    char_vocab = set()
    for line in df["msg"]:
        for c in line:
            char_vocab.add(c)
    return char_vocab

In [540]:
train = pd.read_csv("data/train.csv")
val = pd.read_csv("data/val.csv")
test = pd.read_csv("data/test.csv")

print("train:",train.shape)
print("val:",val.shape)
print("test:",test.shape)

train: (3618, 2)
val: (775, 2)
test: (776, 2)


### 문자 종류 갯수 확인

In [4]:
train_vocab = build_vocab_from_df(train)
val_vocab = build_vocab_from_df(val)
test_vocab = build_vocab_from_df(test)
print(len(train_vocab))
print(sorted(train_vocab))
print(len(val_vocab))
print(sorted(val_vocab))
print(len(test_vocab))
print(sorted(test_vocab))

print(set(test_vocab) - set(train_vocab))
print(set(val_vocab) - set(train_vocab))

94
[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~', '£', '…']
88
[' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '£', '…']
88
[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/

## vocab 생성

In [538]:
char2idx = dict()
char2idx["pad"] = 0
char2idx["unk"] = 1
for index, char in enumerate(train_vocab):
    char2idx[char] = index + 2

idx2char = {value: key for key, value in char2idx.items()}
print(idx2char)
print("vocab size:", len(char2idx))

{0: 'pad', 1: 'unk', 2: 'P', 3: '!', 4: "'", 5: 'L', 6: '"', 7: 'z', 8: 'Z', 9: '.', 10: 'r', 11: '[', 12: 'K', 13: '1', 14: 'w', 15: '…', 16: '&', 17: 'p', 18: 'V', 19: '\\', 20: 'J', 21: '@', 22: '#', 23: 'O', 24: 'S', 25: '?', 26: 'M', 27: '$', 28: 'F', 29: 'b', 30: 'T', 31: 'W', 32: '7', 33: ']', 34: 'G', 35: '=', 36: '5', 37: 'N', 38: 'o', 39: 'B', 40: '%', 41: 'x', 42: '(', 43: 'q', 44: 'D', 45: 'I', 46: 'i', 47: 'C', 48: ',', 49: 'U', 50: ' ', 51: 'h', 52: 'g', 53: 'A', 54: 'c', 55: '9', 56: '~', 57: ':', 58: ')', 59: '*', 60: 'l', 61: '-', 62: 'H', 63: 'y', 64: '|', 65: 'd', 66: 's', 67: '+', 68: 'X', 69: '0', 70: '6', 71: '8', 72: 'v', 73: 'Y', 74: '<', 75: 'k', 76: 'u', 77: '3', 78: 'a', 79: 'f', 80: 'R', 81: 'Q', 82: '4', 83: '_', 84: '/', 85: 'm', 86: 'e', 87: 'n', 88: 'j', 89: '>', 90: '£', 91: '^', 92: 'E', 93: '2', 94: ';', 95: 't'}
vocab size: 96


In [539]:
seq_len = train["msg"].str.len().max()
input_length = len(char2idx)
print(seq_len)

199


In [307]:
class MyDataset(Dataset):
    def __init__(self, data=train):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        x = self.data["msg"][idx]
        y = self.data["label"][idx]

        return x, y

In [308]:
def tokenize(string: str):
    x_data = [char2idx[c] for c in string]
    return torch.tensor(x_data)

In [497]:
def collate_fn(samples):
    x = [i[0] for i in samples]
    x = [tokenize(i) for i in x]
    x = pad_sequence(x, batch_first=True)
    
    y = [i[1] for i in samples]
    y = torch.tensor(y, dtype=torch.float32)

    return x, y

In [499]:
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()

        self.input_dim = input_dim
        self.hidden_size = hidden_size

        self.rnn = nn.RNNCell(input_dim, hidden_size)

    def forward(self, input):
        ht = torch.zeros(self.hidden_size)

        for character in input:
            ht = self.rnn(character, ht)
        return ht


class Net(nn.Module):
    def __init__(self, embedding_dim, hidden_size):
        super(Net, self).__init__()

        self.em = nn.Embedding(len(char2idx), embedding_dim)  # 임베딩
        self.rnn = RNNCell_Encoder(embedding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.em(x)
        x = x.squeeze(0)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return

In [531]:
model = Net(3, 20)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [537]:
def training(epoch, model, trainloader, valloader):
    correct = 0
    total = 0
    running_loss = 0

    model.train()

    for x, y in trainloader:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=0)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()

    epoch_loss = running_loss / len(trainloader)
    epoch_acc = correct / total

    val_correct = 0
    val_total = 0
    val_running_loss = 0

    model.eval()
    with torch.no_grad():
        for x, y in valloader:
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=0)
            val_correct += (y_pred == y).sum().item()
            val_total += y.size(0)
            val_running_loss += loss.item()

    epoch_val_loss = val_running_loss / len(valloader)
    epoch_val_acc = val_correct / val_total

    print(
        f"epoch :{epoch}\t\
         training_loss : {epoch_loss}\t\
         training_accuarcy :{epoch_acc}\t\
         valid_loss : {epoch_val_loss}\t\
         valid_accuracy : {epoch_val_acc}"
    )

    return epoch_loss, epoch_acc, epoch_val_loss, epoch_val_acc

In [536]:
import time

train_ds = MyDataset(train)
val_ds = MyDataset(val)

epochs = 5
train_loss = []
train_acc = []
val_loss = []
val_acc = []

start = time.time()
for epoch in range(epochs):
    epoch_start = time.time()

    train_iterator = iter(
        DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=collate_fn)
    )
    val_iterator = iter(
        DataLoader(val_ds, batch_size=1, shuffle=True, collate_fn=collate_fn)
    )

    epoch_loss, epoch_acc, epoch_val_loss, epoch_val_acc = training(
        epoch, model, train_iterator, val_iterator
    )

    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    val_loss.append(epoch_val_loss)
    val_acc.append(epoch_val_acc)

    epoch_end = time.time()
    print(f"epoch {epoch} finished in {epoch_end-epoch_start:.2f} seconds")

end = time.time()
print(
    f"epoch :{epoch}\t\
        training_loss : {epoch_loss}\t\
        training_accuarcy :{epoch_acc}\t\
        valid_loss : {val_loss}\t\
        valid_accuracy : {val_acc}"
)

print(f"training finished in {end-start:.2f} seconds")

epoch : 0 loss : 0.0 accuarcy : 0.87369 valid_loss : 0.0 valid_accuracy : 0.87355
epoch 0 finished in 22.72 seconds
epoch : 1 loss : 0.0 accuarcy : 0.87369 valid_loss : 0.0 valid_accuracy : 0.87355
epoch 1 finished in 22.64 seconds
epoch : 2 loss : 0.0 accuarcy : 0.87369 valid_loss : 0.0 valid_accuracy : 0.87355
epoch 2 finished in 22.54 seconds
epoch : 3 loss : 0.0 accuarcy : 0.87369 valid_loss : 0.0 valid_accuracy : 0.87355
epoch 3 finished in 22.63 seconds
epoch : 4 loss : 0.0 accuarcy : 0.87369 valid_loss : 0.0 valid_accuracy : 0.87355
epoch 4 finished in 22.36 seconds
training finished in 112.88 seconds


In [530]:
(train["label"] == 0).sum() / train.shape[0]

0.8736871199557766